# **저시력자를 위한 원화 화폐 분류**
---
- 본 과제는 UltraLytics YOLO v5 모델 사용을 권장합니다.
    - 본 파일의 목차는 UltraLytics YOLO v5에 맞게 작성되어 있습니다.
    - 다른 모델을 찾아서 사용하셔도 좋습니다.
    - 산출물이 잘 나오면 됩니다 : )
---

## 0.미션
---
- **과제 수행 목표**
    - 본 과제는 Object Detection 문제입니다.
    - Object Detection 문제로 접근하기 위해 **데이터셋 전처리**를 하셔야 합니다.
    - 데이터셋 : money_dataset.zip
        1. 데이터셋은 압축 파일로 제공됩니다.
        2. 압축 파일 안에는 화폐마다 폴더가 개별적으로 존재합니다.
        3. 폴더 안에는 화폐 이미지와 화폐 정보가 담긴 json 파일이 있습니다.
    - 여러분이 직접 촬영한 화폐 사진들을 탐지 과정에서 이용 해보세요.
    - 이미지에 화폐 하나만 나오게 촬영하는 것은 지양해주세요.
    - 다양한 방법으로 화폐를 촬영하고 결과를 확인해보세요.
        - ex 1) 화폐의 모든 종류를 한 이미지에 나오게 촬영
        - ex 2) 여러 화폐를 겹치게 하여 촬영
---
- **Key Point**
    1. 모델에 맞는 폴더 구조 확인
    2. 이미지 축소 비율에 맞춰 좌표값 변경
        - 좌표를 이미지 리사이즈한 비율로 변경
    3. 모델에 맞는 정보 추출/형식 변경
        - json 파일에서 정보 추출 및 모델 형식에 맞게 변경
    4. 화폐당 하나의 클래스로 변경
        - 총 8개 클래스
    5. 모델 선택 필요
---

## 1.환경설정

### (1) 구글 드라이브 연동, 데이터 다운로드
---
- 아래의 코드 셀을 반드시 실행시켜야 합니다.
---

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
# !pip install gdown

### (2) 데이터셋 불러오기
---
- **세부요구사항**
    - 데이터셋 파일의 압축을 해제하세요.
---
- 예제 코드에서는 zipfile 모듈을 이용하였습니다.
    - [zipfile document](https://docs.python.org/3/library/zipfile.html#zipfile-objects)
    - 해당 모듈 이외에 자신이 잘 알고 있는 방법을 사용해도 됩니다.
---

In [3]:
import zipfile, gdown
from tqdm import tqdm
from pathlib import Path,os
base_path = Path('/content/drive/MyDrive/에이블/미니프로젝트3차')
data_path = base_path / Path('dataset')
raw_path = base_path/Path('raw')
file_name = Path("money_dataset.zip")
money_data_path = base_path / file_name

In [4]:
# import zipfile, gdown,os
# url ="https://drive.google.com/file/d/1k1tXDK35s6BsMTPGWSl5GVGNoPfC898X/view?usp=drive_link"
# file_name = "money_dataset.zip"
# output = "/content/drive/MyDrive/에이블/미니프로젝트3차/" + file_name # 변경 가능
# if not os.path.exists(output):
#     gdown.download(url=url, output=output, quiet=False, fuzzy=True)

In [5]:
# 데이터셋 압축 파일 경로 : 유저별로 상이할 수 있음
money_data = zipfile.ZipFile(money_data_path)

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
# 데이터셋 압축 해제
money_data.extractall('/content/Dataset/')

## 2.데이터 전처리

### (1) 폴더 구조 생성 및 파일 이동
---
- **세부요구사항**
    -  모델에서 요구하는 폴더 구조를 만들어야 합니다.
        - Hint : Image와 Label을 구분하는 폴더를 만들어 주세요
---
- 예제 코드에서는 glob, shutil 모듈을 이용하였습니다.
    - [glob document](https://docs.python.org/3/library/glob.html) | [shutil document](https://docs.python.org/3/library/shutil.html)
    - 해당 모듈 이외에 자신이 잘 알고 있는 방법을 사용해도 됩니다.
---

In [8]:
# 1.폴더 구조 만들기
!mkdir /content/Dataset/images;
!mkdir /content/Dataset/images/train; mkdir /content/Dataset/images/val

!mkdir /content/Dataset/labels;
!mkdir /content/Dataset/labels/train; mkdir /content/Dataset/labels/val

In [9]:
import glob, shutil

In [10]:
# 2. Dataset metadata 입력
won_list = ['10', '50', '100', '500', '1000', '5000', '10000', '50000']
data_path = '/content/Dataset/'

---
- 데이터를 Training set | Validation set으로 분할하세요.
    - 예시 : Training과 Validation은 8:2로 분리
- Hint : 이미지 데이터는 /images에, JSON 데이터는 /labels에 넣어주세요
    - 예시 : /dataset/images/train, /dataset/labels/train
    - 예제 코드에서는 glob, shutil 모듈을 이용하였습니다.
    - [glob document](https://docs.python.org/3/library/glob.html) | [shutil document](https://docs.python.org/3/library/shutil.html)

    ※ 해당 모듈 이외에 자신이 잘 알고 있는 방법을 사용해도 됩니다.
    
---

In [11]:
# jpg, json 파일 분리를 위한 함수 생성
def moving(data):
    for cur in glob.glob(f'/content/Dataset/{data}/*.jpg'):
        shutil.copy(cur, '/content/Dataset/images/train')
    for cur in glob.glob(f'/content/Dataset/{data}/*.json'):
        shutil.copy(cur, '/content/Dataset/labels/train')

# 단위별로 개별 폴더에 있는 파일을 하나하나 들어가 빼내주기 위한 반복문
money=10
while(money<=10000):
    moving(money)
    money*=10
money=50
while(money<=50000):
    moving(money)
    money*=10

# jpg와 json을 나눠서 각각 train데이터에 넣었음

In [12]:
# images-train과 labels-train에 있는 0.2퍼를 images-val과 labels-var로 보냄
# 그럼 images-train이랑 labels-train에는 0.8퍼만 남아있도록 파일 이동
# 3. 데이터를 Training set | Validation set으로 분할하세요.

from sklearn.model_selection import train_test_split
from glob import glob
import shutil
import os

train_img_path = '/content/Dataset/images/train/'
train_lab_path = '/content/Dataset/labels/train/'
val_img_path = '/content/Dataset/images/val/'
val_lab_path = '/content/Dataset/labels/val/'

# 이미지와 라벨 파일 리스트로 가져오기
image_files = os.listdir(train_img_path)
label_files = os.listdir(train_lab_path)

train_img, val_img, train_lab, val_lab = train_test_split(image_files, label_files, test_size=0.2, random_state=2023)

for img_file in train_img:
    shutil.move(os.path.join(train_img_path, img_file), os.path.join(val_img_path, img_file))
for lab_file in train_lab:
    shutil.move(os.path.join(train_lab_path, lab_file), os.path.join(val_lab_path, lab_file))

In [13]:
# 크기 출력
print(len(train_img))
print(len(val_img))
print(len(train_lab))
print(len(val_lab))

4174
1044
4174
1044


### (2) json에서 정보 추출
---
- **세부요구사항**
    - json 파일에서 필요한 정보를 추출하세요:
        - 위치 정보 : x1, x2, y1, y2
        - 박스 정보 : shape_type
        - 클래스 정보 : labels
    - 화폐당 하나의 클래스로 변경하세요.
        - json 파일에는 화폐 클래스가 앞뒷면으로 구분되어 있습니다.
        - 화폐의 앞뒷면 구분을 없애주세요.
            - 예시 : 'ten_front', 'ten_back' -> 'ten'
    - 화폐의 위치 정보를 YOLO 모델 형식에 맞게 변경 해주세요.
        - 사용되는 이미지는 원본에서 1/5로 축소되어 있습니다.
        - json 파일의 정보는 원본 기준 데이터이므로 위치 정보 추출을 할 때 x값과 y값을 1/5로 줄여주세요.
    - 이렇게 변경된 정보를 YOLO label 형식에 맞게 txt파일로 저장 해 주세요.
        - Hint : YOLO Labeling Format [label, x-center, y-center, width-norm, height-norm]
---

In [14]:
import os, json, glob

In [15]:
json_path = '/content/Dataset/labels/'
temp_list = ['train', 'val']

coin = {'Ten' : 0,
        'Fifty' : 1,
        'Hundred' : 2,
        'Five_Hundred' : 3,
        'Thousand' : 4,
        'Five_Thousand' : 5,
        'Ten_Thousand' : 6,
        'Fifty_Thousand' : 7}

In [16]:
labels_train_jsons = glob.glob('/content/Dataset/labels/train/*.json')
labels_val_jsons = glob.glob('/content/Dataset/labels/val/*.json')

In [17]:
 ########################
# 이 셀부터 코드 작성하세요
# Json 파일에서 필요한 정보만 골라 txt로 바꾸는 작업임을 기억하세요!
########################

def change_to_txt(json_file, idx):
    if idx==0:
        cate='train'
    else:
        cate='val'
    for cur in json_file:
        data = json.load(open(cur))
        shapes = data['shapes']
        points = shapes[0]['points']
        label = shapes[0]['label']
        shape = shapes[0]['shape_type']
        x1, y1 = points[0]
        x2, y2 = points[1]
        image_width = data["imageWidth"]
        image_height = data["imageHeight"]

        #label에서 앞, 뒤 제거
        if label[-1]=='k':
            label = label[:len(label)-5]
        else:
            label = label[:len(label)-6]

        #기존에서 1/4한 x, y 센터 찾기
        x1, x2, y1, y2 = x1/5, x2/5, y1/5, y2/5
        width = x2 - x1
        height = y2 - y1
        x_center = x1 + (width/2)
        y_center = y1 + (height/2)

        #전체 이미지에서 0~1사이 값으로 만들기
        x_center = x_center / (image_width/5)
        y_center = y_center / (image_height/5)
        width = width / (image_width/5)
        height = height / (image_height/5)

        #txt 파일로 만들어 저장
        txt_name = data["imagePath"].split(".")[0] + ".txt"
        f = open(f"/content/Dataset/labels/{cate}/{txt_name}", 'w')
        f.write(f'{coin[label]} {x_center} {y_center} {width} {height}')
        f.close()

change_to_txt(labels_train_jsons, 0)
change_to_txt(labels_val_jsons, 1)

### (3) 데이터셋 정보가 담긴 파일 생성
---
- **세부요구사항**
    - 파일 안에 있어야 할 정보는 아래와 같습니다.
        - 학습할 클래스 이름 정보
        - 학습할 클래스 수 정보
        - Training, Validation 데이터셋 위치 정보
---
- 가장 대중적으로 이용하는 라이브러리는 yaml 입니다.
    - [yaml document](https://pyyaml.org/wiki/PyYAMLDocumentation)
    - 해당 모듈 이외에 자신이 잘 알고 있는 방법을 사용해도 됩니다.
---

In [18]:
import yaml

In [19]:
coin_dict = {0:'10', 1:'50', 2:'100', 3:'500', 4:'1000', 5:'5000', 6:'10000', 7:'50000'}

In [20]:
########################
# 이 셀부터 코드 작성하세요
########################
names = list(coin_dict.values())
nc = len(names)
train_path = '/content/Dataset/images/train'
val_path = '/content/Dataset/images/val'

dataset_info = {
    "names": names,
    "nc": nc,
    "train": train_path,
    "val": val_path,
}

with open('/content/Dataset/coin.yaml', 'w') as f :
    yaml.dump(dataset_info, f, default_flow_style=False)

## 3.모델링

### (1) 모델 라이브러리 설치
---

In [ ]:
!git clone https://github.com/ultralytics/yolov5  # clone
!pip install -r yolov5/requirements.txt  # install

In [22]:
# ########################
# # 이 셀부터 코드 작성하세요
# ########################
# temp_str = 'setuptools<=64.0.2\n'

# f = open('/content/yolov5/requirements.txt', 'r')
# f_str = f.readlines()
# f.close()

# f2 = open('/content/yolov5/requirements.txt', 'w')

# for idx, val in enumerate(f_str) :
#     if 'setuptools' in val :
#         idx_v = idx
#         f_str.remove(val)
#         f_str.insert(idx_v, temp_str)

# for val in f_str :
#     f2.write(val)

# f2.close()

In [ ]:
!cd yolov5; pip install -r requirements.txt

### (2) 가중치 파일 다운로드
---
- **세부요구사항**
    - 모델 개발자가 제공하는 사전 학습 가중치 파일을 다운로드 하세요.
        - 해당 과정이 불필요하다면 넘어가셔도 됩니다!
---

In [ ]:
%cd /content/yolov5
!wget https://github.com/ultralytics/yolov5/releases/download/v7.0/yolov5x.pt

### (3) 학습 : train.py
---
- **세부요구사항**
    - UltraLytics YOLO v5에는 아래의 데이터가 필요합니다.
        - 데이터셋 정보가 담긴 yaml 파일
        - 사용하려는 모델 구조에 대한 yaml 파일
        - 사용하려는 모델의 가중치 파일
---

In [25]:
# train.py
%cd /content/yolov5
!python train.py --img 640 --batch 16 --epochs 100 --patience 5 --data /content/Dataset/coin.yaml --cfg ./models/yolov5n.yaml --weights yolov5n.pt --name coin_yolov5n_results

/content/yolov5
WARNING ⚠️ invalid check_version(3.1.37, ) requested, please check values.
train: weights=yolov5n.pt, cfg=./models/yolov5n.yaml, data=/content/Dataset/coin.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=100, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=coin_yolov5n_results, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=5, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
WARNING ⚠️ invalid check_version(5.9.5, ) requested, please check values.
YOLOv5 🚀 v7.0-224-g6262c7f Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla V100-SXM2-16GB, 16151MiB)

hyperparameters: lr0=0.01, lrf=


## 4.탐지 : detect.py
---
- **세부요구사항**
    - 학습 과정에서 생성된 가중치 파일을 이용하세요.
    - IoU threshold를 0.25 이하로 설정하세요.
    - confidence threshold를 0.75 이상으로 설정하세요.
---
- 여러분이 **직접 촬영한 화폐 사진과 동영상**을 탐지 과정에 이용하여 결과를 확인하세요.
    - 조건
        1. 화폐의 수를 늘려가며 촬영 해보세요.
            - ex) 50원 하나, 50원 둘, 50원 셋, ...
        2. 화폐의 종류를 늘려가며 촬영 해보세요.
            - ex) 50원 하나와 100원 하나, 50원 하나와 100원 하나와 1000원 하나, ...
        3. 사진은 최소 30장 이상, 동영상은 최소 하나 이상 촬영하여 사용 해보세요.
---

In [26]:
########################
# 이 셀부터 코드 작성하세요
########################
